In [ ]:
print('test')

In [1]:
import tensorflow as tf

/home/sonam/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.__version__

'1.13.1'

In [3]:
import tensorflow as tf
import os
import sys
import cv2
import numpy as np
import tensorflow.contrib.slim as slim
import tensorflow_hub as hub
from utils import *
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
gpu_options = tf.GPUOptions(allow_growth=True)
config = tf.ConfigProto(gpu_options=gpu_options,log_device_placement=True,allow_soft_placement=True)
import lmdb
%matplotlib inline
import matplotlib.pyplot as plt
import random
import pickle
import collections
print('imported')

imported


# Loading Data

In [ ]:
f30k_path = '/dvmm-filer2/datasets/Groundings/data/flickr30k/'
dict_paths_f30k = [f30k_path+'flickr30k_val.pickle',
                   f30k_path+'flickr30k_test.pickle']

ref_path = '/dvmm-filer2/datasets/Groundings/data/referit/referit_splits/'
lmdb_path_ref = '/dvmm-filer2/datasets/Groundings/data/referit/refclef_data/all_data.lmdb'
dict_path_ref = ref_path  +'test.pickle' 

vg_path = '/dvmm-filer2/datasets/Groundings/data/visualgenome/'
dict_paths_vg = [vg_path+'train.pickle',
                 vg_path+'val.pickle',
                 vg_path+'test.pickle']
lmdb_path_vg = vg_path+'data.lmdb'

print('Loading Flickr30k...')
#####Load Flickr30k

# with open(dict_paths_f30k[0], 'rb') as f:
#     dict_val_f30k = pickle.load(f, encoding='latin1')    
    
with open(dict_paths_f30k[1], 'rb') as f:
    dict_test_f30k = pickle.load(f, encoding='latin1')   
print('Loading ReferIt...')
#####Load ReferIt
with open(dict_path_ref, 'rb') as f:
    dict_test_ref = pickle.load(f, encoding='latin1')
    
lmdb_env_ref = lmdb.open(lmdb_path_ref, map_size=int(1e11), readonly=True, lock=False)
txn_ref = lmdb_env_ref.begin(write=False)

print('Loading Visual Genome...')
#####Load Visual Genome
lmdb_env_vg = lmdb.open(lmdb_path_vg, map_size=int(1e11), readonly=True, lock=False)
txn_vg = lmdb_env_vg.begin(write=False)

# with open(dict_paths_vg[1], 'rb') as f:
#     dict_val_vg = pickle.load(f, encoding='latin1')
#     ids_val_vg = list(dict_val_vg.keys())
    
with open(dict_paths_vg[2], 'rb') as f:
    dict_test_vg = pickle.load(f, encoding='latin1')
    ids_test_vg = list(dict_test_vg.keys())
    
print('Loading data done.')


# Correctness Measure

In [ ]:
# just measures how much is the attention map in the box
def calc_correctness_fk(annot,heatmap,orig_img_shape):
    bbox_dict = heat2bbox(heatmap,orig_img_shape)
    bbox, bbox_norm, bbox_score = filter_bbox(bbox_dict=bbox_dict, order='xyxy')
    bbox_norm_annot = union(annot['bbox_norm'])
    bbox_annot = annot['bbox']
    bbox_norm_pred = union(bbox_norm)
    bbox_correctness = isCorrect(bbox_norm_annot, bbox_norm_pred, iou_thr=.5)
    hit_correctness = isCorrectHit(bbox_annot,heatmap,orig_img_shape)
    att_correctness = attCorrectNess(bbox_annot,heatmap,orig_img_shape)
    return bbox_correctness,hit_correctness, att_correctness

def calc_correctness_rit(annot, heatmap, orig_img_shape):
    bbox_dict = heat2bbox(heatmap,orig_img_shape)
    bbox, bbox_norm, bbox_score = filter_bbox(bbox_dict=bbox_dict, order='xyxy')
    bbox_norm_annot = union(annot['bbox_norm'])
    bbox_annot = union(annot['bbox'])
    bbox_norm_pred = union(bbox_norm)
    bbox_correctness = isCorrect(bbox_norm_annot, bbox_norm_pred, iou_thr=.5)
    hit_correctness = isCorrectHit(bbox_annot, heatmap, orig_img_shape)
    att_correctness = attCorrectNess(bbox_annot,heatmap,orig_img_shape)
    return bbox_correctness, hit_correctness, att_correctness

def calc_correctness_vg(annot, heatmap, orig_img_shape):
    bbox_dict = heat2bbox(heatmap,orig_img_shape)
    bbox, bbox_norm, bbox_score = filter_bbox(bbox_dict=bbox_dict, order='xyxy')
    bbox_norm_annot = union(annot['bbox_norm'])
    bbox_annot = union(annot['bbox'])
    bbox_norm_pred = union(bbox_norm)
    bbox_correctness = isCorrect(bbox_norm_annot, bbox_norm_pred, iou_thr=.5)
    hit_correctness = isCorrectHit(bbox_annot, heatmap, orig_img_shape)
    att_correctness = attCorrectNess(bbox_annot,heatmap,orig_img_shape)
    return bbox_correctness, hit_correctness, att_correctness

In [ ]:
def validate_flickr30k_temp(dict_test):
    cnt_overall = 0
    cat_cnt_overall = {}
    cnt_correct = 0
    cat_cnt_correct = {}
    cnt_correct_hit = 0
    cnt_correct_att = 0
    cat_cnt_correct_hit = {}
    att_correct = 0
    cat_att_correct = {}
    wrd_idx_list = []
    sen_idx_list = []
    cat_lvl_scores = {}
    layer0_words = set()
    layer1_words = set()
    layer3_words = set()
    for k,doc_id in enumerate(dict_test):
        if k>num_tst:
            continue
        img = np.reshape(cv2.resize(dict_test[doc_id]['img'],(299,299)),(1,299,299,3))
        orig_img_shape = dict_test[doc_id]['size']
        sen_batch = list(dict_test[doc_id]['sentences'].keys())
        img_batch = np.repeat(img,len(sen_batch),axis=0)
        tensor_list = [heatmap_w, R_i, R_s, lvl_idx_wrd, lvl_idx_sen, lvl_scr_wrd]
        feed_dict = {input_img: img_batch, text_batch: sen_batch, mode: 'test'}
        qry_heats, qry_scores, sen_score, wrd_idx, sen_idx, lvl_scores = sess.run(tensor_list, feed_dict)
        # Getting embeddings for all sentences
        sen_idx_list.extend(sen_idx) #level selected for each sentence 
        for c,sen in enumerate(sen_batch):
            wrds = sen.split()
            wrd_idx_list.extend(wrd_idx[c,:len(wrds)]) # For each sentence in image for each word the level selected
            temp = wrd_idx[c,:len(wrds)]
            
            for i in range(len(temp)):
                if temp[i]==0:
                    layer0_words.add(wrds[i])
                    
                if temp[i]==1:
                    layer1_words.add(wrds[i])
                    
                if temp[i]==3:
                    layer3_words.add(wrds[i])
                
            for query in dict_test[doc_id]['sentences'][sen]:
                #reject not groundable/acceptable queries
                idx = dict_test[doc_id]['sentences'][sen][query]['idx']
                if len(query.split())==0 or len(idx)==0: # idx where word in sentecne
                    continue
                annot = dict_test[doc_id]['sentences'][sen][query]
                category = list(annot['category'])
                if 'notvisual' in category or len(annot['bbox_norm'])==0:
                    continue
                if not check_percent(union(annot['bbox_norm'])):
                    continue
                #if reaches this point, it is groundable/acceptable
                cnt_overall+=1
                for cat in category:
                    if cat not in cat_cnt_overall:
                        cat_cnt_overall[cat] = 0
                    cat_cnt_overall[cat]+=1    
                
                if np.mean(qry_scores[c,idx])==0:
                    pred = {}
                else:
                    heatmap = np.average(qry_heats[c,idx,:], weights = qry_scores[c,idx], axis=0)
                    # calculate hit, box, attention for query
                    bbox_c, hit_c, att_c = calc_correctness_fk(annot,heatmap,orig_img_shape)
                    
                    cnt_correct += bbox_c
                    cnt_correct_hit += hit_c
                    cnt_correct_att += att_c
                    for cat in category:
                        #per-cat lvl score
                        if cat not in cat_lvl_scores:
                            cat_lvl_scores[cat] = []
                        cat_lvl_scores[cat].append(lvl_scores[c,idx,:])
                        
                        #per-cat bbox acc
                        if cat not in cat_cnt_correct:
                            cat_cnt_correct[cat] = 0
                        cat_cnt_correct[cat] += bbox_c
                        
                        #per-cat hit acc
                        if cat not in cat_cnt_correct_hit:
                            cat_cnt_correct_hit[cat] = 0
                        cat_cnt_correct_hit[cat] += hit_c
                        
                        #per-cat att acc
                        #cat_att_correct
                        if cat not in cat_att_correct:
                            cat_att_correct[cat] = []
                        cat_att_correct[cat].append(att_c)
                    
        var = [k,num_tst,100.*cnt_correct/cnt_overall,100.*cnt_correct_hit/cnt_overall]
        prnt = 'Sample {}/{}, IoU_acc:{:.2f}, Hit_acc:{:.2f} \r'.format(var[0],var[1],var[2],var[3])
        sys.stdout.write(prnt)                
        sys.stdout.flush()
    
    #overall acc
    hit_acc = cnt_correct_hit/cnt_overall
    iou_acc = cnt_correct/cnt_overall
    att_acc = cnt_correct_att/cnt_overall
    tmp = cat_att_correct
    #cat-wise acc
    for cat in cat_cnt_correct:
        cat_cnt_correct[cat]/=cat_cnt_overall[cat]
    for cat in cat_cnt_correct_hit:
        cat_cnt_correct_hit[cat]/=cat_cnt_overall[cat]
    for cat in cat_att_correct:
        cat_att_correct[cat]=np.mean(cat_att_correct[cat])
    
    #return tmp, cat_att_correct,cnt_overall,
        
    return layer0_words, layer1_words,layer3_words,iou_acc,hit_acc,att_acc,wrd_idx_list,sen_idx_list,cat_lvl_scores,cat_cnt_correct,cat_cnt_correct_hit,cat_att_correct


def validate_flickr30k(dict_test):
    cnt_overall = 0
    cat_cnt_overall = {}
    cnt_correct = 0
    cat_cnt_correct = {}
    cnt_correct_hit = 0
    cnt_correct_att = 0
    cat_cnt_correct_hit = {}
    att_correct = 0
    cat_att_correct = {}
    wrd_idx_list = []
    sen_idx_list = []
    cat_lvl_scores = {}
    for k,doc_id in enumerate(dict_test):
        if k>num_tst:
            continue
        img = np.reshape(cv2.resize(dict_test[doc_id]['img'],(299,299)),(1,299,299,3))
        orig_img_shape = dict_test[doc_id]['size']
        sen_batch = list(dict_test[doc_id]['sentences'].keys())
        img_batch = np.repeat(img,len(sen_batch),axis=0)
        tensor_list = [heatmap_w, R_i, R_s, lvl_idx_wrd, lvl_idx_sen, lvl_scr_wrd]
        feed_dict = {input_img: img_batch, text_batch: sen_batch, mode: 'test'}
        qry_heats, qry_scores, sen_score, wrd_idx, sen_idx, lvl_scores = sess.run(tensor_list, feed_dict)
        # Getting embeddings for all sentences
        sen_idx_list.extend(sen_idx) #level selected for each sentence 
        for c,sen in enumerate(sen_batch):
            wrds = sen.split()
            wrd_idx_list.extend(wrd_idx[c,:len(wrds)]) # For each sentence in image for each word the level selected
            for query in dict_test[doc_id]['sentences'][sen]:
                #reject not groundable/acceptable queries
                idx = dict_test[doc_id]['sentences'][sen][query]['idx']
                if len(query.split())==0 or len(idx)==0: # idx where word in sentecne
                    continue
                annot = dict_test[doc_id]['sentences'][sen][query]
                category = list(annot['category'])
                if 'notvisual' in category or len(annot['bbox_norm'])==0:
                    continue
                if not check_percent(union(annot['bbox_norm'])):
                    continue
                #if reaches this point, it is groundable/acceptable
                cnt_overall+=1
                for cat in category:
                    if cat not in cat_cnt_overall:
                        cat_cnt_overall[cat] = 0
                    cat_cnt_overall[cat]+=1    
                
                if np.mean(qry_scores[c,idx])==0:
                    pred = {}
                else:
                    heatmap = np.average(qry_heats[c,idx,:], weights = qry_scores[c,idx], axis=0)
                    # calculate hit, box, attention for query
                    bbox_c, hit_c, att_c = calc_correctness_fk(annot,heatmap,orig_img_shape)
                    
                    cnt_correct += bbox_c
                    cnt_correct_hit += hit_c
                    cnt_correct_att += att_c
                    for cat in category:
                        #per-cat lvl score
                        if cat not in cat_lvl_scores:
                            cat_lvl_scores[cat] = []
                        cat_lvl_scores[cat].append(lvl_scores[c,idx,:])
                        
                        #per-cat bbox acc
                        if cat not in cat_cnt_correct:
                            cat_cnt_correct[cat] = 0
                        cat_cnt_correct[cat] += bbox_c
                        
                        #per-cat hit acc
                        if cat not in cat_cnt_correct_hit:
                            cat_cnt_correct_hit[cat] = 0
                        cat_cnt_correct_hit[cat] += hit_c
                        
                        #per-cat att acc
                        #cat_att_correct
                        if cat not in cat_att_correct:
                            cat_att_correct[cat] = []
                        cat_att_correct[cat].append(att_c)
                    
        var = [k,num_tst,100.*cnt_correct/cnt_overall,100.*cnt_correct_hit/cnt_overall]
        prnt = 'Sample {}/{}, IoU_acc:{:.2f}, Hit_acc:{:.2f} \r'.format(var[0],var[1],var[2],var[3])
        sys.stdout.write(prnt)                
        sys.stdout.flush()
    
    #overall acc
    hit_acc = cnt_correct_hit/cnt_overall
    iou_acc = cnt_correct/cnt_overall
    att_acc = cnt_correct_att/cnt_overall
    tmp = cat_att_correct
    #cat-wise acc
    for cat in cat_cnt_correct:
        cat_cnt_correct[cat]/=cat_cnt_overall[cat]
    for cat in cat_cnt_correct_hit:
        cat_cnt_correct_hit[cat]/=cat_cnt_overall[cat]
    for cat in cat_att_correct:
        cat_att_correct[cat]=np.mean(cat_att_correct[cat])
    
    #return tmp, cat_att_correct,cnt_overall,
        
    return iou_acc,hit_acc,att_acc,wrd_idx_list,sen_idx_list,cat_lvl_scores,cat_cnt_correct,cat_cnt_correct_hit,cat_att_correct

def validate_vg_new(dict_test,txn):
    cnt_overall = 0
    cnt_correct_w = 0
    cnt_correct_hit_w = 0
    cnt_correct_s = 0
    cnt_correct_hit_s = 0
    cnt_correct_att_s = 0
    cnt_correct_att_w = 0
    
    wrd_idx_list = []
    sen_idx_list = []
    for k,doc_id in enumerate(dict_test):
        if k>num_tst:
            continue
        imgbin = txn.get(doc_id.encode('utf-8'))
        if imgbin==None:
            continue
        buff = np.frombuffer(imgbin, dtype='uint8')
        if len(buff) == 0:
            continue
        imgbgr = cv2.imdecode(buff, cv2.IMREAD_COLOR)
        imgrgb = imgbgr[:,:,[2,1,0]]

        img = np.reshape(cv2.resize(imgrgb,(299,299)),(1,299,299,3))
        orig_img_shape = dict_test[doc_id]['size'][:2]
        
        sen_batch = []
        annot_batch = []
        #create batch of queries in a doc_id annotation
        for i,annot in enumerate(dict_test[doc_id]['annotations']):
            sen = annot['query']
            if len(sen.split())==0 or len(annot['bbox_norm'])== 0:
                continue
            if not check_percent(union(annot['bbox_norm'])):
                continue
            if any(b>1 for b in annot['bbox_norm']):
                continue
            sen_batch.append(sen)
            annot_batch.append(annot)
        if len(sen_batch)==0:
            continue
        cnt_overall += len(sen_batch)

        img_batch = np.repeat(img,len(sen_batch),axis=0)
        tensor_list = [heatmap_w,heatmap_s, R_i, R_s, lvl_idx_wrd, lvl_idx_sen, lvl_scr_wrd]
        feed_dict = {input_img: img_batch, text_batch: sen_batch, mode: 'test'}
        qry_heats,qry_heat, qry_scores, sen_score, wrd_idx, sen_idx, lvl_scores = sess.run(tensor_list, feed_dict)
        sen_idx_list.extend(sen_idx) 
        
        for c,sen in enumerate(sen_batch):
            idx = [j for j in range(len(sen.split()))]
            wrds = sen.split()
            wrd_idx_list.extend(wrd_idx[c,:len(wrds)])
            if np.mean(qry_scores[c,idx])==0:
                pred = {}
            else:
                heatmap_wrd = np.average(qry_heats[c,idx,:], weights = qry_scores[c,idx], axis=0)
                heatmap_sen = qry_heat[c,:]
                bbox_c_w,hit_c_w,att_c_w = calc_correctness_vg(annot_batch[c],heatmap_wrd,orig_img_shape)
                bbox_c_s,hit_c_s,att_c_s = calc_correctness_vg(annot_batch[c],heatmap_sen,orig_img_shape)
                cnt_correct_w += bbox_c_w
                cnt_correct_hit_w += hit_c_w
                cnt_correct_s += bbox_c_s
                cnt_correct_hit_s += hit_c_s
                cnt_correct_att_w += att_c_w 
                cnt_correct_att_s += att_c_s

        var = [k,num_tst,cnt_correct_w/cnt_overall,cnt_correct_hit_w/cnt_overall]
        var_s = [cnt_correct_s/cnt_overall,cnt_correct_hit_s/cnt_overall]
        prnt0 = 'Sample {}/{}, IoU_acc_w:{:.2f}, IoU_acc_s:{:.2f}'.format(var[0],var[1],var[2],var_s[0])
        prnt1 = ', Hit_acc_w:{:.2f}, Hit_acc_s:{:.2f} \r'.format(var[3],var_s[1])
        sys.stdout.write(prnt0+prnt1)                
        sys.stdout.flush()

    hit_acc_w = cnt_correct_hit_w/cnt_overall
    iou_acc_w = cnt_correct_w/cnt_overall
    hit_acc_s = cnt_correct_hit_s/cnt_overall
    iou_acc_s = cnt_correct_s/cnt_overall
    att_acc_w = cnt_correct_att_w/cnt_overall
    att_acc_s = cnt_correct_att_s/cnt_overall
    
    return iou_acc_w, hit_acc_w,att_acc_w,iou_acc_s, hit_acc_s, att_acc_s, wrd_idx_list, sen_idx_list


def validate_referit_new(dict_test,txn):
    cnt_overall = 0
    cnt_correct_w = 0
    cnt_correct_hit_w = 0
    cnt_correct_s = 0
    cnt_correct_hit_s = 0
    cnt_correct_att_s = 0
    cnt_correct_att_w = 0
    wrd_idx_list = []
    sen_idx_list = []
    for k,doc_id in enumerate(dict_test):
        if k>num_tst:
            continue
        imgbin = txn.get(doc_id.encode('utf-8'))
        if imgbin==None:
            print ("Image not found")
            continue
        buff = np.frombuffer(imgbin, dtype='uint8')
        if len(buff) == 0:
            print ("Image not found")
            continue
        imgbgr = cv2.imdecode(buff, cv2.IMREAD_COLOR)
        imgrgb = imgbgr[:,:,[2,1,0]]

        img = np.reshape(cv2.resize(imgrgb,(299,299)),(1,299,299,3))
        orig_img_shape = dict_test[doc_id]['size'][:2]

        for i,annot in enumerate(dict_test[doc_id]['annotations']):
            if len(annot['bbox_norm'])== 0:
                continue
            if not check_percent(union(annot['bbox_norm'])):
                continue
            if any(b>1 for b in annot['bbox_norm']):
                continue
            unq_qry = set(annot['query'])
            sen_batch = [sen for sen in unq_qry if 0<len(sen.split())<=50] #only unique queries with 0<length<=50
            img_batch = np.repeat(img,len(sen_batch),axis=0)
            tensor_list = [heatmap_w, heatmap_s,R_i, R_s, lvl_idx_wrd, lvl_idx_sen, lvl_scr_wrd]
            feed_dict = {input_img: img_batch, text_batch: sen_batch, mode: 'test'}
            
            qry_heats, qry_heat, qry_scores, sen_score, wrd_idx, sen_idx, lvl_scores = sess.run(tensor_list, feed_dict)
            sen_idx_list.extend(sen_idx)
            cnt_overall += len(sen_batch)
            for c,sen in enumerate(sen_batch):
                idx = [j for j in range(len(sen.split()))]
                if np.mean(qry_scores[c,idx])==0:
                    pred = {}
                else:
                    wrds = sen.split()
                    wrd_idx_list.extend(wrd_idx[c,:len(wrds)])
                    heatmap_wrd = np.average(qry_heats[c,idx,:], weights = qry_scores[c,idx], axis=0)
                    heatmap_sen = qry_heat[c,:]
                    bbox_c_w, hit_c_w, att_c_w = calc_correctness_rit(annot,heatmap_wrd,orig_img_shape)
                    bbox_c_s, hit_c_s, att_c_s = calc_correctness_rit(annot,heatmap_sen,orig_img_shape)
                    cnt_correct_w += bbox_c_w
                    cnt_correct_hit_w += hit_c_w
                    cnt_correct_s += bbox_c_s
                    cnt_correct_hit_s += hit_c_s
                    cnt_correct_att_w += att_c_w 
                    cnt_correct_att_s += att_c_s
                                     
        var = [k,num_tst,cnt_correct_w/cnt_overall,cnt_correct_hit_w/cnt_overall]
        var_s = [cnt_correct_s/cnt_overall,cnt_correct_hit_s/cnt_overall]
        prnt0 = 'Sample {}/{}, IoU_acc_w:{:.2f}, IoU_acc_s:{:.2f}'.format(var[0],var[1],var[2],var_s[0])
        prnt1 = ', Hit_acc_w:{:.2f}, Hit_acc_s:{:.2f} \r'.format(var[3],var_s[1])
        sys.stdout.write(prnt0+prnt1)                
        sys.stdout.flush()

    hit_acc_w = cnt_correct_hit_w/cnt_overall
    iou_acc_w = cnt_correct_w/cnt_overall
    hit_acc_s = cnt_correct_hit_s/cnt_overall
    iou_acc_s = cnt_correct_s/cnt_overall
    att_acc_w = cnt_correct_att_w/cnt_overall
    att_acc_s = cnt_correct_att_s/cnt_overall
    
    return iou_acc_w, hit_acc_w,att_acc_w,iou_acc_s, hit_acc_s, att_acc_s, wrd_idx_list, sen_idx_list

def attCorrectNess(bbox_annot,heatmap,orig_img_shape):
    H,W = orig_img_shape
    heatmap_resized = cv2.resize(heatmap, (W, H))
    h_s = np.sum(heatmap_resized)
    if h_s==0:
        return 0
    else:
        heatmap_resized /= h_s
    att_correctness = 0
    for bbox in bbox_annot:
        x0,y0,x1,y1=bbox
        att_correctness+=np.sum(heatmap_resized[int(y0):int(y1),int(x0):int(x1)])
    return att_correctness

# Attention

In [ ]:
def attn(e_w,v,e_s):
    ## Inputs: local and global cap and img features ##
    ## Output: Heatmap for each word, Global Heatmap, Attnded Vis features, Corr-vals
    #e: ?xTxD, v: ?xNx4xD, e_bar: ?xD
    with tf.variable_scope('attention'):
        ###word-level###
        #heatmap pool
        h = tf.nn.relu(tf.einsum('bij,bklj->bikl',e_w,v)) #pair-wise ev^T: ?xTxNx4
        #attention
        a = tf.einsum('bijk,bjkl->bilk',h,v) #?xTxDx4 attnded visual reps for each of T words
        #pair-wise score
        a_norm = tf.nn.l2_normalize(a,axis=2)
        e_w_norm = tf.nn.l2_normalize(e_w,axis=2)
        R_ik = tf.einsum('bilk,bil->bik',a_norm,e_w_norm) #cosine for T (words,img_reps) for all pairs
        
        R_ik = tf.identity(R_ik,name='level_score_word')
        R_i = tf.reduce_max(R_ik,axis=-1,name='score_word') #?xT
        #R = tf.log(tf.pow(tf.reduce_sum(tf.exp(gamma_1*R_i),axis=1),1/gamma_1)) #? corrs
        #heatmap
        idx_i = tf.argmax(R_ik,axis=-1,name='level_index_word') #?xT index of the featuremap which maximizes R_i
        with tf.name_scope('summaries'):
            tf.summary.histogram('histogram_w', idx_i)
        ii,jj = tf.meshgrid(tf.range(tf.shape(idx_i)[0]),tf.range(tf.shape(idx_i)[1]),indexing='ij')
        ii = tf.cast(ii,tf.int64)
        jj = tf.cast(jj,tf.int64)
        batch_idx_i = tf.stack([tf.reshape(ii,(-1,)),
                                tf.reshape(jj,(-1,)),
                                tf.reshape(idx_i,(-1,))],axis=1) #?Tx3 indices of argmax
        N0=int(np.sqrt(h.get_shape().as_list()[2]))
        h_max = tf.gather_nd(tf.transpose(h,[0,1,3,2]),batch_idx_i) #?TxN retrieving max heatmaps
        heatmap_wd = tf.reshape(h_max,[tf.shape(h)[0],tf.shape(h)[1],N0,N0],name='heatmap_word')
        heatmap_wd_l = tf.reshape(h,[tf.shape(h)[0],tf.shape(h)[1],N0,N0,tf.shape(h)[3]],name='level_heatmap_word')
        
        ###sentence-level###
        #heatmap pool
        h_s = tf.nn.relu(tf.einsum('bj,blkj->blk',e_s,v)) #pair-wise e_bar*v^T: ?xNx4
        #attention
        a_s = tf.einsum('bjk,bjki->bik',h_s,v) #?xDx4 attnded visual reps for sen.
        #pair-wise score
        a_s_norm = tf.nn.l2_normalize(a_s,axis=1)
        e_s_norm = tf.nn.l2_normalize(e_s,axis=1)
        R_sk = tf.einsum('bik,bi->bk',a_s_norm,e_s_norm) #cosine for (sen,img_reps)
        R_sk = tf.identity(R_sk,name='level_score_sentence')
        R_s = tf.reduce_mean(R_sk,axis=-1,name='score_sentence') #?
        #heatmap
        idx_k = tf.argmax(R_sk,axis=-1,name='level_index_sentence') #? index of the featuremap which maximizes R_i
        with tf.name_scope('summaries'):
            tf.summary.histogram('histogram_s', idx_k)
        ii_k = tf.cast(tf.range(tf.shape(idx_k)[0]),dtype='int64')
        batch_idx_k = tf.stack([ii_k,idx_k],axis=1)
        N0_g=int(np.sqrt(h_s.get_shape().as_list()[1]))
        h_s_max = tf.gather_nd(tf.transpose(h_s,[0,2,1]),batch_idx_k) #?xN retrieving max heatmaps
        heatmap_sd = tf.reshape(h_s_max,[-1,N0_g,N0_g],name='heatmap_sentence')
        heatmap_sd_l = tf.reshape(h_s,[-1,N0_g,N0_g,tf.shape(h)[3]],name='level_heatmap_sentence')
        
    return heatmap_wd, heatmap_sd, R_i, R_s

# Model Loading

In [ ]:
model_name = ['model_Performance_AttnGrnd_VGG_Depth_Max_VG_3by3_maxpool_nlayer4',
             'model_Performance_AttnGrnd_VGG_Depth_Max_VG_3by3_maxpool_nlayer3',
             'model_Performance_AttnGrnd_VGG_Depth_Max_VG_3by3_maxpool_nlayer2',
             'model_Performance_AttnGrnd_VGG_Depth_Max_VG_3by3_maxpool_nlayer1',
             'model_AttnGrnd_VGG_Depth_Max_VG_maxpool',
             'model_AttnGrnd_VGG_Depth_Max_VG_best_hit',
             'model_Performance_AttnGrnd_VGG_Depth_Max_VG_3by3_resize_nlayer1',
             'model_Performance_AttnGrnd_PNAS_Depth_Max_VG_3by3_resize_nlayer1',
               'model_Performance_AttnGrnd_PNAS_Depth_Max_VG_1by1_3by3_resize_nlayer2',
             'model_Performance_AttnGrnd_PNAS_Depth_Max_VG_3by3_resize_nlayer_first2_sec12']
           
print(len(model_name))

In [ ]:
#loading grounding pretrained model
condition = model_name[9]
print('Loading grounding pretrained model...')
model_path = './saved_models/'+ condition
print(model_path)
#model_path = '../model_CNN_avg'
sess, graph = load_model(model_path,config)
print('Model Loaded')

In [ ]:
#tf.all_variables()
#sess.close()

In [ ]:
#heatmap_wd, heatmap_sd, R_i, R_s = attn(e_w,v,e_s)

In [ ]:
# t = sess.graph.get_tensor_by_name("attention/level_heatmap_word:0")
# print(t.shape)

In [ ]:
input_img = sess.graph.get_tensor_by_name("input_img:0")
text_batch = sess.graph.get_tensor_by_name("text_input:0")
mode = sess.graph.get_tensor_by_name("mode:0")
e_w = sess.graph.get_tensor_by_name("w_embedding:0")
e_s = sess.graph.get_tensor_by_name("sen_embedding:0")
v = sess.graph.get_tensor_by_name("stack_v/stacked_image_feature_maps:0")
heatmap_wd, heatmap_sd, R_i, R_s = attn(e_w,v,e_s)
print(R_i.shape)

R_i = sess.graph.get_tensor_by_name("attention/score_word:0")#/transpose_2
R_s = sess.graph.get_tensor_by_name("attention/score_sentence:0")
heatmap_w = sess.graph.get_tensor_by_name("attention/heatmap_word:0")
heatmap_s = sess.graph.get_tensor_by_name("attention/heatmap_sentence:0")

#heatmap_wk = sess.graph.get_tensor_by_name("attention/level_heatmap_word:0")
#heatmap_sk = sess.graph.get_tensor_by_name("attention/level_heatmap_sentence:0")
lvl_idx_wrd = sess.graph.get_tensor_by_name('attention/level_index_word:0') #level mazimising each word T
lvl_scr_wrd = sess.graph.get_tensor_by_name('attention/level_score_word:0') #score for level mazimising each word T
lvl_idx_sen = sess.graph.get_tensor_by_name('attention/level_index_sentence:0')#level mazimising sentence
#lvl_idx_wrd = sess.graph.get_tensor_by_name('attention/ArgMax:0')
#lvl_scr_wrd = sess.graph.get_tensor_by_name('attention/einsum_2/transpose_2:0')
#lvl_idx_sen = sess.graph.get_tensor_by_name('attention/ArgMax_1:0')
print('Loading done.')

# Evaluation Flickr

In [ ]:
# print(len(dict_test_f30k))
# print(dict_test_f30k['4842876705'])
#print(dict_test_f30k['4842876705']['sentences'].keys())
#print(dict_test_f30k['4842876705']['sentences']['A man in a blue shirt and jeans plays bass on the street .']['A man'])

In [ ]:
#temp
#### Flickr30k ####
print('Test on Flickr30k\n')
num_tst = len(dict_test_f30k)
num_tst = 100
layer0, layer1,layer3, iou_acc_f30k, hit_acc_f30k, att_acc_f30k, wrd_idx_list_f30k,sen_idx_list_f30k,cat_lvl_scores_f30k,cat_iou_acc_f30k,cat_hit_acc_f30k,cat_att_acc_f30k = validate_flickr30k_temp(dict_test_f30k)
print('done testing')
print('flickr')
print('wrds for layer0', layer0)
print('wrds for layer1', layer1)
print(collections.Counter(wrd_idx_list_f30k))
print(collections.Counter(sen_idx_list_f30k))
print(cat_lvl_scores_f30k['clothing'][2])
print('Cat_acc\n', cat_iou_acc_f30k)
print('Cat hit\n', cat_hit_acc_f30k)
print('Cat att\n', cat_att_acc_f30k)
print('iou_acc',iou_acc_f30k)
print('hit_acc',hit_acc_f30k)
print('att_acc',att_acc_f30k)
print('start saving')
# with open('./evaluations/train_vg/test_f30k/'+condition+'.pickle', 'wb') as f:
#     pickle.dump({'iou_acc':iou_acc_f30k,
#                  'hit_acc':hit_acc_f30k,
#                  'att_acc':att_acc_f30k,
#                  'wrd_idx_list':wrd_idx_list_f30k,
#                  'sen_idx_list':sen_idx_list_f30k,
#                  'cat_lvl_scores':cat_lvl_scores_f30k,
#                  'cat_iou_acc':cat_iou_acc_f30k,
#                  'cat_hit_acc':cat_hit_acc_f30k,
#                  'cat_att_acc':cat_att_acc_f30k},f,protocol=pickle.HIGHEST_PROTOCOL)
print('done')

In [ ]:
print('0',layer0)
print('1',layer1)
print('3',layer3)

In [ ]:
#### Flickr30k ####
print('Test on Flickr30k\n')
num_tst = len(dict_test_f30k)
num_tst = 50
iou_acc_f30k, hit_acc_f30k, att_acc_f30k, wrd_idx_list_f30k,sen_idx_list_f30k,cat_lvl_scores_f30k,cat_iou_acc_f30k,cat_hit_acc_f30k,cat_att_acc_f30k = validate_flickr30k(dict_test_f30k)
print('done testing')
print('flickr')
print(collections.Counter(wrd_idx_list_f30k))
print(collections.Counter(sen_idx_list_f30k))
print(cat_lvl_scores_f30k['clothing'][2])
print('Cat_acc\n', cat_iou_acc_f30k)
print('Cat hit\n', cat_hit_acc_f30k)
print('Cat att\n', cat_att_acc_f30k)
print('iou_acc',iou_acc_f30k)
print('hit_acc',hit_acc_f30k)
print('att_acc',att_acc_f30k)
print('start saving')
# with open('./evaluations/train_vg/test_f30k/'+condition+'.pickle', 'wb') as f:
#     pickle.dump({'iou_acc':iou_acc_f30k,
#                  'hit_acc':hit_acc_f30k,
#                  'att_acc':att_acc_f30k,
#                  'wrd_idx_list':wrd_idx_list_f30k,
#                  'sen_idx_list':sen_idx_list_f30k,
#                  'cat_lvl_scores':cat_lvl_scores_f30k,
#                  'cat_iou_acc':cat_iou_acc_f30k,
#                  'cat_hit_acc':cat_hit_acc_f30k,
#                  'cat_att_acc':cat_att_acc_f30k},f,protocol=pickle.HIGHEST_PROTOCOL)
print('done')

In [ ]:
# !pip install jupyter_contrib_nbextensions
# !jupyter contrib nbextension install --user

# # Evaluation ReferIt

In [ ]:
#print(dict_test_ref['14730']['captions'])
#print(dict_test_ref['14730']['annotations'][1])

In [ ]:
print('Test on ReferIt\n')
#### ReferIt ####
num_tst = len(dict_test_ref)
iou_acc_ref_w, hit_acc_ref_w,att_acc_ref_w, iou_acc_ref_s, hit_acc_ref_s,att_acc_ref_s, wrd_idx_list_ref, sen_idx_list_ref = validate_referit_new(dict_test_ref,txn_ref)
print('referit')
print(collections.Counter(wrd_idx_list_ref))
print(collections.Counter(sen_idx_list_ref))
print('iou_acc_word',iou_acc_ref_w)
print('iou_acc_sent',iou_acc_ref_s)
print('hit_acc_word',hit_acc_ref_w)
print('hit_acc_sent',hit_acc_ref_s)
print('att_acc_sent',att_acc_ref_s)
print('att_acc_word',att_acc_ref_w)
with open('./evaluations/train_vg/test_referit/'+condition+'.pickle', 'wb') as f:
    pickle.dump({'iou_acc_w':iou_acc_ref_w,
                 'hit_acc_w':hit_acc_ref_w,
                  'att_acc_w':att_acc_ref_w,
                 'iou_acc_s':iou_acc_ref_s,
                 'hit_acc_s':hit_acc_ref_s,
                 'att_acc_s':att_acc_ref_s,
                 'wrd_idx_list':wrd_idx_list_ref,
                 'sen_idx_list':sen_idx_list_ref},f,protocol=pickle.HIGHEST_PROTOCOL)
print('All Done')

# Evaluation VisualGenome

In [ ]:
print(condition)

In [ ]:
#print(dict_test_vg['2345650']['annotations'][0])

In [ ]:
print('Test on Visual Genome\n')
#### Visual Genome ####
num_tst = len(dict_test_vg)
iou_acc_vg_w, hit_acc_vg_w, att_acc_vg_w, iou_acc_vg_s, hit_acc_vg_s, att_acc_vg_s, wrd_idx_list_vg ,sen_idx_list_vg = validate_vg_new(dict_test_vg,txn_vg)
print('All done.')
print('visualgenome')
print(collections.Counter(wrd_idx_list_vg))
print(collections.Counter(sen_idx_list_vg))
print('iou_acc_word',iou_acc_vg_w)
print('iou_acc_sent',iou_acc_vg_s)
print('hit_acc_word',hit_acc_vg_w)
print('hit_acc_sent',hit_acc_vg_s)
print('att_acc_sent',att_acc_vg_s)
print('att_acc_word',att_acc_vg_w)
with open('./evaluations/train_vg/test_vg/'+condition+'.pickle', 'wb') as f:
    pickle.dump({'iou_acc_w':iou_acc_vg_w,
                 'hit_acc_w':hit_acc_vg_w,
                  'att_acc_w':att_acc_vg_w,
                 'iou_acc_s':iou_acc_vg_s,
                 'hit_acc_s':hit_acc_vg_s,
                 'att_acc_s':att_acc_vg_s,
                 'wrd_idx_list':wrd_idx_list_vg,
                 'sen_idx_list':sen_idx_list_vg},f,protocol=pickle.HIGHEST_PROTOCOL)
print('AllDone')

In [ ]:
print('sonam')

# Evaluating All

In [ ]:
print('Test on Flickr30k\n')
num_tst = len(dict_test_f30k)
iou_acc_f30k, hit_acc_f30k, att_acc_f30k, wrd_idx_list_f30k,sen_idx_list_f30k,cat_lvl_scores_f30k,cat_iou_acc_f30k,cat_hit_acc_f30k,cat_att_acc_f30k = validate_flickr30k(dict_test_f30k)

print('done testing')
print('flickr')
print(collections.Counter(wrd_idx_list_f30k))
print(collections.Counter(sen_idx_list_f30k))
print(cat_lvl_scores_f30k['clothing'][2])
print('Cat_acc\n', cat_iou_acc_f30k)
print('Cat hit\n', cat_hit_acc_f30k)
print('Cat att\n', cat_att_acc_f30k)
print('iou_acc',iou_acc_f30k)
print('hit_acc',hit_acc_f30k)
print('att_acc',hit_acc_f30k)
print('start saving')
with open('./evaluations/train_pnas/test_f30k/'+condition+'.pickle', 'wb') as f:
    pickle.dump({'iou_acc':iou_acc_f30k,
                 'hit_acc':hit_acc_f30k,
                 'att_acc':att_acc_f30k,
                 'wrd_idx_list':wrd_idx_list_f30k,
                 'sen_idx_list':sen_idx_list_f30k,
                 'cat_lvl_scores':cat_lvl_scores_f30k,
                 'cat_iou_acc':cat_iou_acc_f30k,
                 'cat_hit_acc':cat_hit_acc_f30k,
                 'cat_att_acc':cat_att_acc_f30k},f,protocol=pickle.HIGHEST_PROTOCOL)
print('done')


print('Test on ReferIt\n')
#### ReferIt ####
num_tst = len(dict_test_ref)
iou_acc_ref_w, hit_acc_ref_w,att_acc_ref_w, iou_acc_ref_s, hit_acc_ref_s,att_acc_ref_s, wrd_idx_list_ref, sen_idx_list_ref = validate_referit_new(dict_test_ref,txn_ref)
print('referit')
print(collections.Counter(wrd_idx_list_ref))
print(collections.Counter(sen_idx_list_ref))
print('iou_acc_word',iou_acc_ref_w)
print('iou_acc_sent',iou_acc_ref_s)
print('hit_acc_word',hit_acc_ref_w)
print('hit_acc_sent',hit_acc_ref_s)
print('att_acc_sent',att_acc_ref_s)
print('att_acc_word',att_acc_ref_w)

with open('./evaluations/train_vg/test_referit/'+condition+'.pickle', 'wb') as f:
    pickle.dump({'iou_acc_w':iou_acc_ref_w,
                 'hit_acc_w':hit_acc_ref_w,
                  'att_acc_w':att_acc_ref_w,
                 'iou_acc_s':iou_acc_ref_s,
                 'hit_acc_s':hit_acc_ref_s,
                 'att_acc_s':att_acc_ref_s,
                 'wrd_idx_list':wrd_idx_list_ref,
                 'sen_idx_list':sen_idx_list_ref},f,protocol=pickle.HIGHEST_PROTOCOL)
print('All Done')

print('Test on Visual Genome\n')
#### Visual Genome ####
num_tst = len(dict_test_vg)
iou_acc_vg_w, hit_acc_vg_w, att_acc_vg_w, iou_acc_vg_s, hit_acc_vg_s, att_acc_vg_s, wrd_idx_list_vg ,sen_idx_list_vg = validate_vg_new(dict_test_vg,txn_vg)
print('All done.')
print('visualgenome')
print(collections.Counter(wrd_idx_list_vg))
print(collections.Counter(sen_idx_list_vg))
print('iou_acc_word',iou_acc_vg_w)
print('iou_acc_sent',iou_acc_vg_s)
print('hit_acc_word',hit_acc_vg_w)
print('hit_acc_sent',hit_acc_vg_s)
print('att_acc_sent',att_acc_vg_s)
print('att_acc_word',att_acc_vg_w)
with open('./evaluations/train_pnas/test_vg/'+condition+'.pickle', 'wb') as f:
    pickle.dump({'iou_acc_w':iou_acc_vg_w,
                 'hit_acc_w':hit_acc_vg_w,
                  'att_acc_w':att_acc_vg_w,
                 'iou_acc_s':iou_acc_vg_s,
                 'hit_acc_s':hit_acc_vg_s,
                 'att_acc_s':att_acc_vg_s,
                 'wrd_idx_list':wrd_idx_list_vg,
                 'sen_idx_list':sen_idx_list_vg},f,protocol=pickle.HIGHEST_PROTOCOL)
print('AllDone')

In [ ]:
print('ready')

# Testing

In [ ]:
# def calc_idx(cat_lvl_scores,cats,method):
#     cat_scr = {}
#     cat_idx = {}
#     cat_avg = {}
#     n_lvl = 4
#     final = [[' ']*(len(cats)+1) for _ in range((2*n_lvl))]
#     final[0][0] = method
#     for i,cat in enumerate(cats):
#         cat_scr[cat]=[]
#         cat_idx[cat]=[0]*n_lvl
#         for scrs in cat_lvl_scores[cat]:
#             cat_scr[cat].extend(scrs)
#         tmp = np.argmax(cat_scr[cat],axis=1)
#         for t in tmp:
#             cat_idx[cat][t]+=1
#         cat_avg[cat] = np.mean(cat_scr[cat],axis=0)
#         for j in range(2*n_lvl):
#             if j<n_lvl:
#                 final[j][i+1] = cat_idx[cat][j]
#             else:
#                 final[j][i+1] = cat_avg[cat][j-n_lvl]
#     return final

In [ ]:
#final = calc_idx(acc_dict['cat_lvl_scores'],m_rows_cat_idx[0][1:],e_f.split('/')[-1])

In [ ]:
# cat_n={}
# for k_ in dict_test_f30k:
#     for sen in dict_test_f30k[k_]['sentences']:
#         for qry in dict_test_f30k[k_]['sentences'][sen]:
#             for cat in dict_test_f30k[k_]['sentences'][sen][qry]['category']:
#                 if cat=='notvisual':
#                     continue
#                 if cat not in cat_n:
#                     cat_n[cat]=0
#                 cat_n[cat]+=1

# Gather and save

In [ ]:
# cat_n={}
# qry_n=0
# for doc_id in dict_test_f30k:
#     for sen in dict_test_f30k[doc_id]['sentences']:
#         for query in dict_test_f30k[doc_id]['sentences'][sen]:
#             idx = dict_test_f30k[doc_id]['sentences'][sen][query]['idx']
#             if len(query.split())==0 or len(idx)==0:
#                 continue
#             annot = dict_test_f30k[doc_id]['sentences'][sen][query]
#             category = annot['category']
#             if 'notvisual' in category or len(annot['bbox_norm'])==0:
#                 continue
#             if not check_percent(union(annot['bbox_norm'])):
#                 continue
                    
#             for cat in category:
#                 if cat=='notvisual':
#                     continue
#                 if cat not in cat_n:
#                     cat_n[cat]=0
#                 cat_n[cat]+=1
#                 qry_n+=1

In [ ]:
# print(qry_n)
# cat_n

In [ ]:
# def calc_idx(cat_lvl_scores,cats,method):
#     cat_scr = {}
#     cat_idx = {}
#     cat_avg = {}
#     n_lvl = 4
#     final = [[' ']*(len(cats)+1) for _ in range((2*n_lvl))]
#     final[0][0] = method
#     for i,cat in enumerate(cats):
#         cat_scr[cat]=[]
#         cat_idx[cat]=[0]*n_lvl
#         for scrs in cat_lvl_scores[cat]:
#             cat_scr[cat].extend(scrs)
#         tmp = np.argmax(cat_scr[cat],axis=1)
#         for t in tmp:
#             cat_idx[cat][t]+=1
#         cat_avg[cat] = np.mean(cat_scr[cat],axis=0)
#         for j in range(2*n_lvl):
#             if j<n_lvl:
#                 final[j][i+1] = cat_idx[cat][j]
#             else:
#                 final[j][i+1] = cat_avg[cat][j-n_lvl]
#     return final


# Checking

In [ ]:
print('test')


In [ ]:
import glob
import pandas as pd
import collections
train = ['vg','pnas']
test = ['f30k','referit','vg']
#test = ['f30k']
pre_path = './evaluations/train_'
cols = ['iou_acc_w','iou_acc_s','hit_acc_w','hit_acc_s','att_acc_w','att_acc_s','Layer0_Word','Layer1_Word',\
                    'Layer2_Word','Layer3_Word','Layer0_Sen','Layer1_Sen', 'Layer2_Sen','Layer3_Sen']
df = pd.DataFrame(columns = cols)
tmp_cols = ['iou_acc','iou_acc','hit_acc','hit_acc','att_acc','att_acc']
names =[]
counter = 0
for train_ in train:
    print(train_)
    for test_ in test:
        print(test_ + 'started')
        dir_path = pre_path + train_ + '/test_' + test_+ '/*pickle'
        files = glob.glob(dir_path)
        #print(files)
        ####print(files)
        for f in files:
            file = open(f,'rb')
            data = pickle.load(file)
            
            name = train_ + test_+ ' '+ f.split('/')[-1].split('_')[-2] + f.split('/')[-1].split('_')[-1]
            name = name.split('.')[0]
            #print(name)
            names.append(name)
            stats = []
            #stats.append(name)
            wrd_counter = collections.Counter(data['wrd_idx_list'])
            sen_counter = collections.Counter(data['sen_idx_list'])
            
            if test_=='f30k':
                #print(tmp_cols)
                #print(data)
                for x in tmp_cols[0:6]: 
                    if x in data:
                        stats.append(round(data[x]*100,2))
                    else:
                        stats.append(0)
            else:
                for x in cols[0:6]:
                    if x in data:
                        stats.append(round(data[x]*100,2))
                    else:
                        stats.append(0)
                    
            for i in range(0,4):
                if i in wrd_counter:
                    stats.append(wrd_counter[i])
                else:
                    stats.append(0)
            for i in range(0,4):
                if i in sen_counter:
                    stats.append(int(sen_counter[i]))
                else:
                    stats.append(int(0))
            df.loc[counter] = stats
            counter += 1

df.index = names
df['name'] = names
df  
df.to_csv('SummaryVGTrain.csv')
df
# for nlayer1 accuracy

In [ ]:
# m_rows
# m_rows_cat_hit
# m_rows_cat_att

In [ ]:
# myFile = open('./evaluations/cat_wise_lvl_idx.csv', 'w')  
# with myFile:
#     writer = csv.writer(myFile)
#     writer.writerows(m_rows_cat_idx)
    
# myFile = open('./evaluations/cat_wise_hit.csv', 'w')  
# with myFile:
#     writer = csv.writer(myFile)
#     writer.writerows(m_rows_cat_hit)
    
# myFile = open('./evaluations/cat_wise_att.csv', 'w')  
# with myFile:
#     writer = csv.writer(myFile)
#     writer.writerows(m_rows_cat_att)
    
# myFile = open('./evaluations/overall.csv', 'w')  
# with myFile:
#     writer = csv.writer(myFile)
#     writer.writerows(m_rows_l)

# Visualization

In [ ]:
#mention level
from textwrap import wrap
from matplotlib.backends.backend_pdf import PdfPages
from utils import img_heat_bbox_disp

pdf = PdfPages("VGG_Depth_VG_on_Flickr30k_Level.pdf")
for k,doc_id in enumerate(dict_test):
    prnt = 'Sample {}/{} \r'.format(k,len(dict_test))
    sys.stdout.write(prnt)                
    sys.stdout.flush()
    img = np.reshape(cv2.resize(dict_test[doc_id]['img'],(299,299)),(1,299,299,3))
    orig_img_shape = dict_test[doc_id]['size']
    sen_batch = list(dict_test[doc_id]['sentences'].keys())
    img_batch = np.repeat(img,len(sen_batch),axis=0)
    tensor_list = [heatmap_w, heatmap_s, R_i, R_s, lvl_idx_wrd, lvl_idx_sen]
    feed_dict = {input_img: img_batch, text_batch: sen_batch, mode: 'test'}
    qry_heats, sen_heat, qry_scores, sen_score, w_idx, sen_idx = sess.run(tensor_list, feed_dict)
    for c,sen in enumerate(sen_batch):
        if c>0:
            continue
        title_str = sen#+',  Sen-Img Score: %.2f'%sen_score
        title = "\n".join(wrap(title_str, 120))
        heatmap = sen_heat[c]
        fig = img_heat_bbox_disp(img[0,:], heatmap, title=title, en_name=None, bboxes=[], order='xyxy',show=False,dot_max=True)
        pdf.savefig(fig)
        for query in dict_test[doc_id]['sentences'][sen]:
            #reject not groundable/acceptable queries
            idx = dict_test[doc_id]['sentences'][sen][query]['idx']
            if len(query.split())==0 or len(idx)==0:
                continue
            annot = dict_test[doc_id]['sentences'][sen][query]
            category = annot['category']
            if 'notvisual' in category or len(annot['bbox_norm'])==0:
                continue
            if not check_percent(union(annot['bbox_norm'])):
                continue
            #if reaches this point, it is groundable/acceptable

            if np.mean(qry_scores[c,idx])==0:
                continue
            else:
                heatmap = np.average(qry_heats[c,idx,:], weights = qry_scores[c,idx], axis=0)
                strs_idx = [str(s) for s in w_idx[c,idx]]
                en_txt = query+', %.2f'%np.mean(qry_scores[c,idx])+', '+','.join(strs_idx)
                fig = img_heat_bbox_disp(img[0,:], heatmap, title=title, en_name=en_txt, bboxes=annot['bbox_norm'], order='xyxy', show=False,dot_max=True)
                pdf.savefig(fig)
pdf.close()

In [ ]:
pdf.close()

In [ ]:
num_tst = len(dict_test)
iou_acc_f,hit_acc_f = validate_flickr30k(dict_test)

# Feature Level Histogram

In [ ]:
#VGG on VG
plt.hist(idx_wrd, bins=10)
plt.hist(idx_sen, bins=5)

In [ ]:
#VGG on MSCOCO
plt.hist(idx_wrd, bins=10)
plt.hist(idx_sen, bins=5)

In [ ]:
#PNASNET on VG
plt.hist(idx_wrd, bins=10)
plt.hist(idx_sen, bins=10)

In [ ]:
words = []
idx_wrd = []
idx_sen = []
idx_wrd_tf = sess.graph.get_tensor_by_name('attention/ArgMax:0')
idx_sen_tf = sess.graph.get_tensor_by_name('attention/ArgMax_1:0')
for k,doc_id in enumerate(dict_test):
    prnt = 'Sample {}/{} \r'.format(k,len(dict_test))
    sys.stdout.write(prnt)                
    sys.stdout.flush()
    img = np.reshape(cv2.resize(dict_test[doc_id]['img'],(299,299)),(1,299,299,3))
    orig_img_shape = dict_test[doc_id]['size']
    sen_batch = list(dict_test[doc_id]['sentences'].keys())
    img_batch = np.repeat(img,len(sen_batch),axis=0)
    tensor_list = [idx_wrd_tf, idx_sen_tf]
    feed_dict = {input_img: img_batch, text_batch: sen_batch, mode: 'test'}
    idx_max_w, idx_max_s = sess.run(tensor_list, feed_dict)
    idx_sen.extend(idx_max_s)
    for c,sen in enumerate(sen_batch):
        wrds = sen.split()
        words.extend(wrds)
        idx_wrd.extend(idx_max_w[c,:len(wrds)])

In [ ]:
!tensorboard --logdir ../logs/vgg/vg --port=8993